In [1]:
import random

In [2]:
import model
import selfplay
import game
import config

In [3]:
def get_probs_with_log(self, g: game.GameState):
    "runs N simulations and returns visit probabilities at root"
    root = selfplay.Node(g)
    for i in range(config.num_simulate):
        self.simulate(root)
    self.print_tree(root)
    return [
        root.children[i].visit/(root.visit-1) if i in root.children else 0.0
        for i in range(config.num_actions)
    ]

def my_print_tree(self, n: selfplay.Node, depth=0, all=True):
    if n.visit:
        a = ">>"
    else:
        a = "--"
    if depth < 3 and (all or n.visit):
        print(a + 4*depth*" " + f"board position: {repr(n.board)}, prob: {n.prob:0.2}, value: {n.value:0.2},"
                            f" visit: {n.visit}, value_sum: {n.value_sum:0.2}")
        for ch in n.children.values():
            self.print_tree(ch, depth+1)

selfplay.MCTS.print_tree = my_print_tree
selfplay.MCTS.get_probs = get_probs_with_log

In [4]:
net = model.Model()
net.load()
mcts = selfplay.MCTS(net)

In [5]:
import chess

In [9]:
g = game.GameState()
# g.board = chess.Board('3k4/8/8/8/4q3/2b5/8/2K5 b - - 9 286')
g.board = chess.Board('3k4/8/8/8/4q3/2b5/8/2K5 b - - 9 286')  # mate in 2
# g.board = chess.Board('3k4/8/8/8/8/2b5/4q3/1K6 b - - 11 287')  # mate in 1

In [10]:
config.num_simulate = 1000

In [11]:
p = mcts.get_probs(g)

>>board position: Board('3k4/8/8/8/4q3/2b5/8/2K5 b - - 9 286'), prob: 0.0, value: 0.0093, visit: 1000, value_sum: -1.7e+02
>>    board position: Board('3k4/8/8/8/4q3/8/8/b1K5 w - - 10 287'), prob: 0.023, value: 0.0093, visit: 20, value_sum: 0.15
>>        board position: Board('3k4/8/8/8/4q3/8/8/b2K4 b - - 11 287'), prob: 0.5, value: 0.0094, visit: 9, value_sum: -0.065
>>        board position: Board('3k4/8/8/8/4q3/8/3K4/b7 b - - 11 287'), prob: 0.5, value: 0.0092, visit: 10, value_sum: -0.072
>>    board position: Board('3k4/8/8/8/4q3/8/8/2K1b3 w - - 10 287'), prob: 0.023, value: 0.0092, visit: 19, value_sum: 0.14
>>        board position: Board('3k4/8/8/8/4q3/8/8/3Kb3 b - - 11 287'), prob: 0.5, value: 0.0092, visit: 9, value_sum: -0.064
>>        board position: Board('3k4/8/8/8/4q3/8/1K6/4b3 b - - 11 287'), prob: 0.5, value: 0.0092, visit: 9, value_sum: -0.064
>>    board position: Board('3k4/8/8/8/4q3/8/1b6/2K5 w - - 10 287'), prob: 0.023, value: 0.0094, visit: 19, value_sum: 0.12


: 

In [90]:
b = chess.Board('3k4/8/8/8/8/2b5/1q6/1K6 w - - 12 288')

In [92]:
g1 = game.GameState()
g1.board = b

In [93]:
g1.leaf_value()

1

In [11]:
g = game.GameState()
history = []
move_history = []

while not g.terminated():
    probs = mcts.get_probs(g)
    print(f"{sum(probs)=}")
    #print(vis(str(g.to_image())))
    history.append([g, probs, 0])
    # choose action acc to probs
    action = random.choices(list(range(config.num_actions)), probs)[0]
    print(f"{action=}")
    move_history.append(action)
    g = g.next_state(action)

print(f"g={g.to_image()} {g.terminated()=}")
#print(vis(str(g.to_image())))


sum(probs)=1.0
action=536
sum(probs)=1.0
action=3234
sum(probs)=1.0
action=853
sum(probs)=1.0
action=4013
sum(probs)=1.0
action=658
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/krkartikay/code/AlphaZeroFinal/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_15361/3597355656.py", line 6, in <module>
    probs = mcts.get_probs(g)
            ^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_15361/2070827925.py", line 5, in get_probs_with_log
    self.simulate(root)
  File "/home/krkartikay/code/AlphaZeroFinal/selfplay.py", line 121, in simulate
    val_child = - self.simulate(m)
                  ^^^^^^^^^^^^^^^^
  File "/home/krkartikay/code/AlphaZeroFinal/selfplay.py", line 121, in simulate
    val_child = - self.simulate(m)
                  ^^^^^^^^^^^^^^^^
  File "/home/krkartikay/code/AlphaZeroFinal/selfplay.py", line 126, in simulate
    n.expand(self.net)
  File "/home/krkartikay/code/AlphaZeroFinal/selfplay.py", line -1, in expand
KeyboardInterrupt

During handling of the above exception, another

In [ ]:
# log game outcome
winner = g.winner()
# print(move_history, winner)d
# fill in the value function
if winner != 0:
    for state in history:
        g = state[0]
        if g.player() == winner:
            state[2] = -1
        else:
            state[2] = 1